# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#Read csv
weather_data_df = pd.read_csv('weather_data.csv')

weather_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Tiksi,71.6872,128.8694,17.49,91,83,2.42,RU,1633825667
1,Marsá Maţrūḩ,31.3525,27.2453,71.58,74,0,4.99,EG,1633825812
2,Bredasdorp,-34.5322,20.0403,47.52,93,1,3.44,ZA,1633825814
3,Bluff,-46.6000,168.3333,56.23,57,58,3.24,NZ,1633825815
4,Butaritari,3.0707,172.7902,81.64,74,39,11.52,KI,1633825817


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
#Create variables to hold lat & lng and humidity data
locations = weather_data_df[["Lat", "Lng"]].astype(float)
humidity_data = weather_data_df['Humidity'].astype(float)

#Create gmaps figure
fig = gmaps.figure()

#Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_data, 
                                 dissipating=False, max_intensity=humidity_data.max(),
  
                                 point_radius = 1)
#Add heat layer on top of the gmaps
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#Create a new dataframe that has cities narrowed down to fit the specified weather conditions
weather_criteria = weather_data_df.loc[(weather_data_df["Wind Speed"] <= 10) & (weather_data_df["Cloudiness"] == 0) & \
                                   (weather_data_df["Max Temp"] >= 70) & (weather_data_df["Max Temp"] <= 80)].dropna()
weather_criteria

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,Marsá Maţrūḩ,31.3525,27.2453,71.58,74,0,4.99,EG,1633825812
169,São Filipe,14.8961,-24.4956,79.56,76,0,7.36,CV,1633826061
194,Chechat,24.7667,75.8833,76.12,65,0,4.83,IN,1633826102
200,Koutsouras,35.0339,25.9469,70.11,79,0,1.74,GR,1633826111
227,Riyadh,24.6877,46.7219,73.54,18,0,2.57,SA,1633826150
290,Bhīnmāl,25.0000,72.2500,79.45,77,0,6.58,IN,1633826244
305,Alexandria,31.2156,29.9553,74.19,72,0,8.75,EG,1633826244
316,Tura,25.5198,90.2201,75.38,86,0,2.57,IN,1633826283
366,Poum,-20.2333,164.0167,74.34,65,0,9.26,NC,1633826356
412,Hirado,33.3597,129.5531,78.15,66,0,1.23,JP,1633826425


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
#Craete a new dataframe that will store the hotel name closest to each of the city
hotel_df = weather_criteria.loc[:, ['City', 'Country', 'Lat', 'Lng']]
hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Marsá Maţrūḩ,EG,31.3525,27.2453,
169,São Filipe,CV,14.8961,-24.4956,
194,Chechat,IN,24.7667,75.8833,
200,Koutsouras,GR,35.0339,25.9469,
227,Riyadh,SA,24.6877,46.7219,
290,Bhīnmāl,IN,25.0000,72.2500,
305,Alexandria,EG,31.2156,29.9553,
316,Tura,IN,25.5198,90.2201,
366,Poum,NC,-20.2333,164.0167,
412,Hirado,JP,33.3597,129.5531,


In [7]:
#Set up base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Configure parameters
params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # Retrieve lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]

    # add keyword to params dict
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try: 
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = np.NaN

hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Marsá Maţrūḩ,EG,31.3525,27.2453,Beau Site
169,São Filipe,CV,14.8961,-24.4956,Hotel Xaguate
194,Chechat,IN,24.7667,75.8833,NaN
200,Koutsouras,GR,35.0339,25.9469,South Coast Hotel
227,Riyadh,SA,24.6877,46.7219,TULIP INN
290,Bhīnmāl,IN,25.0000,72.2500,Hotel Sai Palace
305,Alexandria,EG,31.2156,29.9553,TOLIP Hotel Alexandria
316,Tura,IN,25.5198,90.2201,Hotel C&C
366,Poum,NC,-20.2333,164.0167,NaN
412,Hirado,JP,33.3597,129.5531,Yukai Resort Hotel Ranpu


In [9]:
#Drop cities with no hotels near by 
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
1,Marsá Maţrūḩ,EG,31.3525,27.2453,Beau Site
169,São Filipe,CV,14.8961,-24.4956,Hotel Xaguate
200,Koutsouras,GR,35.0339,25.9469,South Coast Hotel
227,Riyadh,SA,24.6877,46.7219,TULIP INN
290,Bhīnmāl,IN,25.0000,72.2500,Hotel Sai Palace
305,Alexandria,EG,31.2156,29.9553,TOLIP Hotel Alexandria
316,Tura,IN,25.5198,90.2201,Hotel C&C
412,Hirado,JP,33.3597,129.5531,Yukai Resort Hotel Ranpu


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))